In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
import numpy as np
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define the Encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.resnet = models.resnet18(weights=None)
        self.resnet.fc = nn.Identity()

    def forward(self, x):
        return self.resnet(x)

# Define the Pneumonia Classifier
class PneumoniaClassifier(nn.Module):
    def __init__(self, encoder, num_classes=2):
        super(PneumoniaClassifier, self).__init__()
        self.encoder = encoder
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        features = self.encoder(x)
        logits = self.fc(features)
        return logits

In [4]:
# Custom Dataset for RSNA
class RSNADataset(Dataset):
    def __init__(self, metadata_df, images_dir, target_size=(224, 224)):
        self.metadata_df = metadata_df
        self.images_dir = images_dir
        self.target_size = target_size
        
        # Get all image files
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
        
        # Create mapping from patientId to image path
        self.patient_to_image = {}
        unmatched_patients = set()
        
        for patient_id in metadata_df['patientId']:
            matches = [f for f in self.image_files if patient_id in f]
            
            if len(matches) == 1:
                self.patient_to_image[patient_id] = os.path.join(images_dir, matches[0])
            elif len(matches) > 1:
                print(f"Warning: Multiple matches for patient {patient_id}")
                self.patient_to_image[patient_id] = os.path.join(images_dir, matches[0])
            else:
                unmatched_patients.add(patient_id)
        
        self.valid_indices = [
            i for i, patient_id in enumerate(metadata_df['patientId']) 
            if patient_id in self.patient_to_image
        ]
        
        if not self.valid_indices:
            raise ValueError("No matching images found for any patient!")
            
        print(f"Found {len(self.valid_indices)} valid patients out of {len(metadata_df)}")
        if unmatched_patients:
            print(f"{len(unmatched_patients)} patients without matching images")

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(target_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        idx = self.valid_indices[idx]
        row = self.metadata_df.iloc[idx]
        patient_id = row['patientId']
        
        img_path = self.patient_to_image[patient_id]
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Failed to load image: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        
        label = row['Target']
        return img, label


In [5]:
# Load the pre-trained model
checkpoint_path = '/kaggle/input/simclr_but_better/pytorch/default/1/ssl_model_final (1).pth'
checkpoint = torch.load(checkpoint_path, map_location=device)

In [6]:
# Load and split the training data
base_path = '/kaggle/input/rsna-pneumonia-processed-dataset'
train_metadata_df = pd.read_csv(os.path.join(base_path, 'stage2_train_metadata.csv'))
train_images_dir = os.path.join(base_path, 'Training', 'Images')

# Split into train (60%), validation (20%), and test (20%)
train_df, temp_df = train_test_split(train_metadata_df, test_size=0.4, random_state=42, stratify=train_metadata_df['Target'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['Target'])

# Create datasets
train_dataset = RSNADataset(train_df, train_images_dir)
val_dataset = RSNADataset(val_df, train_images_dir)
test_dataset = RSNADataset(test_df, train_images_dir)

# Create dataloaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

Found 18136 valid patients out of 18136
Found 6045 valid patients out of 6045
Found 6046 valid patients out of 6046


In [7]:
# Initialize the encoder
encoder = Encoder()
encoder_state_dict = {k.replace('encoder.', ''): v for k, v in checkpoint['model_state_dict'].items() 
                     if k.startswith('encoder.')}
encoder.load_state_dict(encoder_state_dict)
encoder = encoder.to(device)

# Instantiate the classifier
model = PneumoniaClassifier(encoder).to(device)
for param in model.encoder.parameters():
    param.requires_grad = False


In [8]:
# Class weights
labels = train_metadata_df['Target'].values
class_weights = torch.tensor(compute_class_weight('balanced', classes=[0,1], y=labels), 
                           dtype=torch.float).to(device)
print(f"Class weights: {class_weights}")

Class weights: tensor([0.7311, 1.5817], device='cuda:0')


In [9]:
# Training setup
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [10]:
from tqdm import tqdm

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0
    
    # Wrap train_loader with tqdm for progress bar
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for batch_idx, (images, labels) in enumerate(train_loader_tqdm):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar description with current loss
        train_loader_tqdm.set_postfix(loss=loss.item(), acc=100. * correct / total)
        
        # Print detailed loss every 20 batches
        if batch_idx % 20 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}, Acc: {100. * correct / total:.2f}%")
    
    train_acc = 100. * correct / total
    print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")
    
    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        # Wrap val_loader with tqdm for progress bar
        val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)
        for images, labels in val_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            val_loader_tqdm.set_postfix(loss=val_loss/(batch_idx+1), acc=100. * correct / total)
    
    val_acc = 100. * correct / total
    print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.2f}%\n")
    scheduler.step()

Epoch 1/20:   1%|▏         | 2/142 [00:07<07:27,  3.20s/it, acc=62.5, loss=0.587]

Epoch 1/20, Batch 0, Loss: 0.6750, Acc: 50.00%


Epoch 1/20:  15%|█▌        | 22/142 [00:32<03:06,  1.55s/it, acc=73.7, loss=0.468]

Epoch 1/20, Batch 20, Loss: 0.5767, Acc: 73.47%


Epoch 1/20:  30%|██▉       | 42/142 [00:59<02:22,  1.43s/it, acc=75, loss=0.436]  

...

Epoch 20/20, Batch 100, Loss: 0.4617, Acc: 79.66%


Epoch 20/20:  85%|████████▌ | 121/142 [02:22<00:32,  1.53s/it, acc=79.4, loss=0.441]

Epoch 20/20, Batch 120, Loss: 0.4411, Acc: 79.40%


Epoch 20/20, Batch 140, Loss: 0.4172, Acc: 79.48%

Epoch 20/20 Summary:
Train Loss: 0.4388, Train Acc: 79.48%


Val Loss: 0.4394, Val Acc: 78.88%



In [11]:
# Test evaluation
model.eval()
test_loss, correct, total = 0, 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100. * correct / total
print(f"\nFinal Test Results - Loss: {test_loss/len(test_loader):.4f}, Accuracy: {test_acc:.2f}%")

# Save model
torch.save(model.state_dict(), 'pneumonia_classifier_final.pth')
print("Model saved to pneumonia_classifier_final.pth")


Final Test Results - Loss: 0.4279, Accuracy: 80.22%
Model saved to pneumonia_classifier_final.pth
